# Mapping

In [2]:
import pandas as pd

In [3]:
wos_docs = pd.read_csv("datasets/wos_articles.csv")
scop_docs = pd.read_csv("datasets/Training_data_docs_scp.csv")
gosch_docs = pd.read_csv("datasets/Articles_Dataset.csv")

In [4]:
# Afficher les informations sur les DataFrames
print(wos_docs.info())
print(scop_docs.info())
print(gosch_docs.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Titre de l’article  181 non-null    object 
 1   Mots-clés           176 non-null    object 
 2   Résumé              181 non-null    object 
 3   issn                181 non-null    object 
 4   Titre de journal    181 non-null    object 
 5   sjr_score           160 non-null    float64
dtypes: float64(1), object(5)
memory usage: 8.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5340 entries, 0 to 5339
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   issn             5340 non-null   object 
 1   title            5340 non-null   object 
 2   abstract         5132 non-null   object 
 3   author keywords  4558 non-null   object 
 4   Nom de la revue  5340 non-null   object 
 5   Score SJR     

- Renommer les colonnes des DataFrames et Convertir les colonnes en types de données appropriés

In [5]:
# Renommer les colonnes de wos_docs
wos_docs.columns = ['title', 'author_keywords', 'abstract', 'issn', 'journal_name', 'sjr_score']

# Renommer les colonnes de scop_docs
scop_docs.columns = ['issn', 'title', 'abstract', 'author_keywords', 'journal_name', 'sjr_score']

# Renommer les colonnes de gosch_docs
gosch_docs.columns = ['title', 'journal_name', 'abstract', 'issn']

In [6]:
# Pour wos_docs
wos_docs = wos_docs.astype({
    'title': 'string',
    'author_keywords': 'string',
    'abstract': 'string',
    'issn': 'string',
    'journal_name': 'string',
    'sjr_score' : 'float'
})

# Pour scop_docs
scop_docs = scop_docs.astype({
    'issn': 'string',
    'title': 'string',
    'abstract': 'string',
    'author_keywords': 'string',
    'journal_name': 'string',
    'sjr_score' : 'float'
})

# Pour gosch_docs
gosch_docs = gosch_docs.astype({
    'title': 'string',
    'journal_name': 'string',
    'abstract': 'string',
    'issn': 'string'
})

- Filtrer les lignes où 'sjr_score' n'est pas égal à 'N/A' et n'est pas NaN et l'ajustement pour sjr_score > 1

In [7]:
wos_docs = wos_docs[(wos_docs['sjr_score'] != 'N/A') & (wos_docs['sjr_score'].notna()) & (wos_docs['sjr_score'] < 1)]
scop_docs = scop_docs[(scop_docs['sjr_score'] != 'N/A') & (scop_docs['sjr_score'].notna()) & (scop_docs['sjr_score'] < 1)]

- Combiner les dataFrames en un seul dataFrame

In [8]:
df_merged = pd.concat([wos_docs,scop_docs,gosch_docs],ignore_index=True)
df_merged

,title,author_keywords,abstract,issn,journal_name,sjr_score
0,Oscillations induced by quiescent adult female...,"bifurcation-analysis , population-dynamics , ...",Aedes aegypti (ae. aegypti: mosquito) is a kno...,19371632,Discrete and continuous dynamical systems-seri...,0.541
1,Reducing the interprocessors migrations of the...,"real time scheduling , ekg , semi-partitionin...","In this work, we consider the scheduling probl...",18951767,Scalable computing-practice and experience,0.308
2,Stig: a generic intelligent tutoring system a ...,"intelligent tutoring system , multi-agents sy...",In this paper is described the design of an in...,18761100,Proceedings of the mediterranean conference on...,0.147
3,Determining the potato samples processed manne...,"inorganic fertilizers , mineral-content , sof...","Through several studies, electronic nose (e-no...",09570233,Measurement science and technology,0.523
4,An intelligent machine learning based routing ...,"ad hoc networks , clustering-algorithm , prot...","Today, vehicular ad-hoc networks (vanet) have ...",21693536,Ieee access,0.960
...,...,...,...,...,...,...
8304,Unsupervised VAD method based on short-time en...,<NA>,Voice Activity Detection (VAD) distinguishes s...,09528091,International Journal of Computer Applications...,NaN
8305,Advances in Machine Intelligence and Computer ...,<NA>,In computer science and in the field of comput...,"00167037, 0046564X",Geochimica et Cosmochimica Acta,NaN
8306,IGAEM: Improved Genetic Algorithm based Entity...,<NA>,The presence of duplicate records is a major d...,"17426588, 17426596",Journal of Physics: Conference Series,NaN
8307,Synonym reduction in a comparative study of cl...,<NA>,"The growing use of smart phones, social networ...",-,International Conference on Information Techno...,NaN


In [9]:
df_merged.duplicated().sum()

732

In [10]:
df_merged.drop_duplicates(inplace=True)

In [11]:
df_merged.duplicated().sum()

0

- Compléter les valeurs manquantes en prenant celles du même titre

In [12]:
df_merged['author_keywords'] = df_merged.groupby('title')['author_keywords'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
df_merged['sjr_score'] = df_merged.groupby('issn')['sjr_score'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
df_merged['abstract'] = df_merged.groupby('title')['abstract'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
df_merged

C:\Users\Bouali\AppData\Local\Temp\ipykernel_15572\1473217197.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged['author_keywords'] = df_merged.groupby('title')['author_keywords'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
C:\Users\Bouali\AppData\Local\Temp\ipykernel_15572\1473217197.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged['sjr_score'] = df_merged.groupby('issn')['sjr_score'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
C:\Users\Bouali\AppData\Local\Temp\ipykernel_15572\1473217197.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged['abstract'] = df_merged.groupby('title')['abstract'].transform(lambda x: x.fillna(method='ffill').fill

,title,author_keywords,abstract,issn,journal_name,sjr_score
0,Oscillations induced by quiescent adult female...,"bifurcation-analysis , population-dynamics , ...",Aedes aegypti (ae. aegypti: mosquito) is a kno...,19371632,Discrete and continuous dynamical systems-seri...,0.541
1,Reducing the interprocessors migrations of the...,"real time scheduling , ekg , semi-partitionin...","In this work, we consider the scheduling probl...",18951767,Scalable computing-practice and experience,0.308
2,Stig: a generic intelligent tutoring system a ...,"intelligent tutoring system , multi-agents sy...",In this paper is described the design of an in...,18761100,Proceedings of the mediterranean conference on...,0.147
3,Determining the potato samples processed manne...,"inorganic fertilizers , mineral-content , sof...","Through several studies, electronic nose (e-no...",09570233,Measurement science and technology,0.523
4,An intelligent machine learning based routing ...,"ad hoc networks , clustering-algorithm , prot...","Today, vehicular ad-hoc networks (vanet) have ...",21693536,Ieee access,0.960
...,...,...,...,...,...,...
8304,Unsupervised VAD method based on short-time en...,"Arabic speech, computer applications, short-ti...",Voice Activity Detection (VAD) distinguishes s...,09528091,International Journal of Computer Applications...,0.313
8305,Advances in Machine Intelligence and Computer ...,<NA>,In computer science and in the field of comput...,"00167037, 0046564X",Geochimica et Cosmochimica Acta,NaN
8306,IGAEM: Improved Genetic Algorithm based Entity...,<NA>,The presence of duplicate records is a major d...,"17426588, 17426596",Journal of Physics: Conference Series,NaN
8307,Synonym reduction in a comparative study of cl...,"Clustering, Dimension reduction, Synonym, Text...","The growing use of smart phones, social networ...",-,International Conference on Information Techno...,NaN


In [13]:
df_merged.isna().sum()

title                 0
author_keywords    4508
abstract             93
issn                  0
journal_name          0
sjr_score          3692
dtype: int64

In [14]:
from rapidfuzz.fuzz import ratio

def similarity_score(string1, string2):
    string1, string2 = string1.lower(), string2.lower()
    # Calculate similarity score using rapidfuzz
    return ratio(string1, string2) / 100.0

In [15]:
# pip install rapidfuzz

In [26]:
from rapidfuzz import fuzz

score = fuzz.ratio("chat", "chaton")
print(score)  # Exemple : 77


80.0


In [16]:
def find_duplicates(dataframe, column, threshold=0.85):
    duplicates = []
    checked = set()
    for i, title1 in enumerate(dataframe[column]):
        for j, title2 in enumerate(dataframe[column]):
            if i != j and (j, i) not in checked:
                score = similarity_score(title1, title2)
                if score >= threshold:
                    duplicates.append((i, j, score))
                checked.add((i, j))
    return duplicates

In [17]:
# Find duplicates based on 'title' with a similarity threshold of 0.85
df_merged.reset_index(inplace=True)
duplicates = find_duplicates(df_merged, 'title', threshold=0.85)

In [18]:
duplicates

[(0, 1645, 1.0),
 (0, 1648, 0.9045226130653267),
 (0, 3391, 0.9045226130653267),
 (0, 3392, 1.0),
 (0, 5127, 1.0),
 (1, 1305, 1.0),
 (2, 1307, 1.0),
 (5, 718, 1.0),
 (6, 2319, 1.0),
 (6, 7428, 0.9906542056074766),
 (8, 253, 0.9859154929577465),
 (8, 7549, 0.9859154929577465),
 (9, 135, 1.0),
 (10, 2283, 1.0),
 (11, 1843, 1.0),
 (14, 1005, 0.9863013698630136),
 (14, 3381, 0.9863013698630136),
 (15, 404, 0.9939393939393939),
 (15, 3380, 0.9759036144578314),
 (16, 474, 1.0),
 (18, 7436, 1.0),
 (19, 7434, 1.0),
 (23, 457, 1.0),
 (23, 7514, 1.0),
 (29, 471, 1.0),
 (29, 7435, 0.993288590604027),
 (47, 51, 0.8585858585858586),
 (55, 998, 0.9369369369369369),
 (55, 1000, 0.8502415458937197),
 (58, 1021, 1.0),
 (59, 1020, 1.0),
 (62, 281, 0.978102189781022),
 (63, 291, 0.989247311827957),
 (64, 292, 0.992),
 (66, 398, 1.0),
 (67, 459, 1.0),
 (67, 7450, 1.0),
 (68, 7531, 1.0),
 (69, 7439, 1.0),
 (70, 491, 0.9317507418397626),
 (70, 7484, 1.0),
 (74, 77, 0.8571428571428572),
 (75, 84, 0.858638743

In [19]:
def map_abstract(dataframe, abstract_column, duplicates):
    for idx1, idx2, score in duplicates:
        original_idx = min(idx1, idx2)
        duplicate_idx = max(idx1, idx2)
        # Check if the original's abstract is NaN or empty
        if pd.isna(dataframe.at[original_idx, abstract_column]) or dataframe.at[original_idx, abstract_column] == '':
            # If so, take the abstract from the duplicate row
            duplicate_abstract = dataframe.at[duplicate_idx, abstract_column]
            if not pd.isna(duplicate_abstract) and duplicate_abstract != '':
                dataframe.at[original_idx, abstract_column] = duplicate_abstract
    return dataframe

def map_journal(dataframe, journal_column, duplicates):
    for idx1, idx2, score in duplicates:
        original_idx = min(idx1, idx2)
        duplicate_idx = max(idx1, idx2)
        # Check if the original's abstract is NaN or empty
        if pd.isna(dataframe.at[original_idx, journal_column]) or dataframe.at[original_idx, journal_column] == '':
            # If so, take the abstract from the duplicate row
            duplicate_journal_name = dataframe.at[duplicate_idx, journal_column]
            if not pd.isna(duplicate_journal_name) and duplicate_journal_name != '':
                dataframe.at[original_idx, journal_column] = duplicate_journal_name
    return dataframe

def map_keywords(dataframe, keywords_column, duplicates):
    for idx1, idx2, score in duplicates:
        original_idx = min(idx1, idx2)
        duplicate_idx = max(idx1, idx2)
        # Check if the duplicate's keywords are NaN or empty
        if pd.isna(dataframe.at[duplicate_idx, keywords_column]) or dataframe.at[duplicate_idx, keywords_column] == '':
            # If so, take the keywords from the original row
            duplicate__keywords = dataframe.at[original_idx, keywords_column]
            if not pd.isna(duplicate__keywords) and duplicate__keywords != '':
                dataframe.at[duplicate_idx, keywords_column] = duplicate__keywords
    return dataframe

def map_sjr(dataframe, sjr_column, duplicates):
    for idx1, idx2, score in duplicates:
        original_idx = min(idx1, idx2)
        duplicate_idx = max(idx1, idx2)
        # Check if the duplicate's sjr_score is NaN or empty
        if pd.isna(dataframe.at[duplicate_idx, sjr_column]) or dataframe.at[duplicate_idx, sjr_column] == '':
            # If so, take the sjr_score from the original row
            duplicate__sjr = dataframe.at[original_idx, sjr_column]
            if not pd.isna(duplicate__sjr):
                dataframe.at[duplicate_idx, sjr_column] = duplicate__sjr
    return dataframe

In [20]:
df_merged = map_abstract(df_merged,"abstract",duplicates)

In [21]:
df_merged = map_journal(df_merged,"journal_name",duplicates)

In [22]:
df_merged = map_sjr(df_merged, 'sjr_score', duplicates)

In [23]:
df_merged

,index,title,author_keywords,abstract,issn,journal_name,sjr_score
0,0,Oscillations induced by quiescent adult female...,"bifurcation-analysis , population-dynamics , ...",Aedes aegypti (ae. aegypti: mosquito) is a kno...,19371632,Discrete and continuous dynamical systems-seri...,0.541
1,1,Reducing the interprocessors migrations of the...,"real time scheduling , ekg , semi-partitionin...","In this work, we consider the scheduling probl...",18951767,Scalable computing-practice and experience,0.308
2,2,Stig: a generic intelligent tutoring system a ...,"intelligent tutoring system , multi-agents sy...",In this paper is described the design of an in...,18761100,Proceedings of the mediterranean conference on...,0.147
3,3,Determining the potato samples processed manne...,"inorganic fertilizers , mineral-content , sof...","Through several studies, electronic nose (e-no...",09570233,Measurement science and technology,0.523
4,4,An intelligent machine learning based routing ...,"ad hoc networks , clustering-algorithm , prot...","Today, vehicular ad-hoc networks (vanet) have ...",21693536,Ieee access,0.960
...,...,...,...,...,...,...,...
7572,8304,Unsupervised VAD method based on short-time en...,"Arabic speech, computer applications, short-ti...",Voice Activity Detection (VAD) distinguishes s...,09528091,International Journal of Computer Applications...,0.313
7573,8305,Advances in Machine Intelligence and Computer ...,<NA>,In computer science and in the field of comput...,"00167037, 0046564X",Geochimica et Cosmochimica Acta,NaN
7574,8306,IGAEM: Improved Genetic Algorithm based Entity...,<NA>,The presence of duplicate records is a major d...,"17426588, 17426596",Journal of Physics: Conference Series,0.180
7575,8307,Synonym reduction in a comparative study of cl...,"Clustering, Dimension reduction, Synonym, Text...","The growing use of smart phones, social networ...",-,International Conference on Information Techno...,0.215


In [24]:
df_merged.to_csv("merged_documents_datasets.csv")